### Mobility processing
This notebook works through the process of adapting Google mobility data into the required format for the model.

In [ ]:
import pandas as pd

from emutools.tex import DummyTexDoc, remove_underscore_multiindexcol
from aust_covid.constants import MOBILITY_MAP, MOBILITY_AVERAGE_WINDOW, set_project_base_path
from aust_covid.inputs import get_raw_state_mobility
from aust_covid.mobility import get_non_wa_mob_averages, get_relative_mobility, map_mobility_locations
from aust_covid.plotting import plot_state_mobility, plot_processed_mobility

project_paths = set_project_base_path("../")

In [ ]:
dummy_doc = DummyTexDoc()

In [ ]:
state_data, jurisdictions, mob_locs = get_raw_state_mobility(dummy_doc)
plot_state_mobility(state_data, jurisdictions, mob_locs)

In [ ]:
mapping_table = pd.DataFrame(MOBILITY_MAP)
mapping_table.index = mapping_table.index.str.replace("_", " ").str.capitalize()
mapping_table.columns = mapping_table.columns.str.replace("_", " ").str.capitalize()
mapping_table = mapping_table.applymap(str)
mapping_table

In [ ]:
wa_data = state_data.loc[state_data["sub_region_1"] == "Western Australia", mob_locs]
state_averages = get_non_wa_mob_averages(state_data, mob_locs, jurisdictions, dummy_doc, cross_ref=False)
non_wa_relmob = get_relative_mobility(state_averages)
wa_relmob = get_relative_mobility(wa_data)
processed_mob = map_mobility_locations(wa_relmob, non_wa_relmob, dummy_doc)
smoothed_mob = processed_mob.rolling(MOBILITY_AVERAGE_WINDOW).mean().dropna()
squared_mob = smoothed_mob ** 2.0

In [ ]:
mobility_types = {
    "original": remove_underscore_multiindexcol(processed_mob),
    "smoothed": remove_underscore_multiindexcol(smoothed_mob),
    "smoothed squared": remove_underscore_multiindexcol(squared_mob),
}
plot_processed_mobility(mobility_types)